# Intro to pyMMS

This notebook will describe how to use the pymms, a package for working with data from the Magnetospheric Multiscale (MMS) mission. Data and mission details can be found at the [Science Data Center](https://lasp.colorado.edu/mms/sdc/public/).

## Installing Packages with Conda
Since I am using `conda` and spacepy is not packaged for `conda`, I need to get `conda` to recognize pip packages. To do so, I follow the [`conda` tutorial](https://conda.io/docs/user-guide/tasks/manage-environments.html) for managing packages.
```bash
conda create --name mmsenv
conda install -n mmsenv spyder numpy scipy matplotlib tqdm
source activate mmsenv # Windos: activate mmsenv
pip install spacepy
```

Note that SciPy and SpacePy require a Fortran compiler to be instealled. [This tutorial](http://skipperkongen.dk/2012/04/27/how-to-install-gfortran-on-mac-os-x/) explains how to install gFortran for MaxOSX. It is available via `Homebrew` and `MacPorts`. What I did was download the DMG file from [GNU](http://gcc.gnu.org/wiki/GFortranBinaries#MacOS). In install Spyder so that it can be launched while in the environment, as per [this page](https://stackoverflow.com/questions/30170468/how-to-run-spyder-in-virtual-environment).

## Installing packages with pip
To get started, we need to install [`numpy`](http://www.numpy.org/), [`scipy`](https://www.scipy.org/index.html), [`spacepy`](https://pythonhosted.org/SpacePy/index.html), [`matplotlib`](https://matplotlib.org/), and [`tqdm`](https://pypi.python.org/pypi/tqdm). I follow the "Installing Via Pip" section on the [SciPy install page](https://www.scipy.org/install.html).
```bash
pip3 install numpy scipy spacepy matplotlib tqdm
```

## Installing CDF
Following the steps
1. Go to the [CDF Download page](https://cdf.gsfc.nasa.gov/html/sw_and_docs.html)
2. Click on the latest initial release (v3.6.4 at the time of this writing)
3. Follow links for your operating system. Mine is [MacOSX](https://cdf.gsfc.nasa.gov/html/sw_and_docs.html)
4. Read the AAREADME.txt
5. Download tarball and install

From the terminal in MacOSX:
```bash
cd ~/Downloads/
wget -v -nd -np -nH https://spdf.sci.gsfc.nasa.gov/pub/software/cdf/dist/cdf36_4/macosx/cdf36_4_0-setup_universal_binary.tar.gz
gunzip cdf36_4_0-setup_universal_binary.tar.gz
tar -vxf cdf36_4_0-setup_universal_binary.tar

# Installs into /Applications/cdf/cdf36_4-dist
open CDF3640ub.pkg
rm CDF3640ub.pkg
```

## Setting PYTHONPATH
Next, I add my python directory to the `$PYTHONPATH` system variable. I do this programmatically here, but it can also be done [in a python or environment startup script](https://stackoverflow.com/questions/3402168/permanently-add-a-directory-to-pythonpath).

In [1]:
import sys
sys.path.append('/Users/argall/Documents/Python/')
sys.path

['',
 '/Users/argall/anaconda3/lib/python36.zip',
 '/Users/argall/anaconda3/lib/python3.6',
 '/Users/argall/anaconda3/lib/python3.6/lib-dynload',
 '/Users/argall/anaconda3/lib/python3.6/site-packages',
 '/Users/argall/anaconda3/lib/python3.6/site-packages/aeosa',
 '/Users/argall/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/Users/argall/.ipython',
 '/Users/argall/Documents/Python/']

## Working with pyMMS
Now I can import the MMS SDC module and create an instance of the MrMMS_SDC_API class.

### Available Files
To see which files are available and motivate the attributes of the MrMMS_SDC_API class, I demonstrate the ``FileNames`` method.

In [2]:
import pymms as mms
mysdc = mms.MrMMS_SDC_API(start_date='2017-07-11', end_date='2017-07-11',
                          data_root='/Users/argall/MrWebData/mms/')
files = mysdc.FileNames()
print(*files, sep='\n')

mms/data/mms4/scm/srvy/l2/scsrvy/2017/07/mms4_scm_srvy_l2_scsrvy_20170711_v2.2.0.cdf
mms/data/mms4/mec/srvy/l2/ephts04d/2017/07/mms4_mec_srvy_l2_ephts04d_20170711_v2.0.2.cdf
mms/data/mms4/mec/srvy/l2/epht89q/2017/07/mms4_mec_srvy_l2_epht89q_20170711_v2.0.2.cdf
mms/data/mms4/mec/srvy/l2/epht89d/2017/07/mms4_mec_srvy_l2_epht89d_20170711_v2.0.2.cdf
mms/data/mms4/hpca/srvy/l2/moments/2017/07/mms4_hpca_srvy_l2_moments_20170711000000_v3.2.5.cdf
mms/data/mms4/hpca/srvy/l2/ion/2017/07/mms4_hpca_srvy_l2_ion_20170711000000_v3.2.5.cdf
mms/data/mms4/fgm/srvy/l2/2017/07/mms4_fgm_srvy_l2_20170711_v5.95.0.cdf
mms/data/mms4/feeps/srvy/l2/ion/2017/07/mms4_feeps_srvy_l2_ion_20170711000000_v6.0.1.cdf
mms/data/mms4/feeps/srvy/l2/electron/2017/07/mms4_feeps_srvy_l2_electron_20170711000000_v6.0.1.cdf
mms/data/mms4/epd-eis/srvy/l2/phxtof/2017/07/mms4_epd-eis_srvy_l2_phxtof_20170711_v4.0.3.cdf
mms/data/mms4/epd-eis/srvy/l2/extof/2017/07/mms4_epd-eis_srvy_l2_extof_20170711_v4.0.3.cdf
mms/data/mms4/epd-eis/srvy

MMS directories and file names have the format:
* **files**: `sc_instr_mode_level[_optdesc]_tstart_vX.Y.Z.cdf`
* **directories**:
  * **srvy**: `sc/instr/mode/level[/optdesc]/YYYY/MM/`
  * **brst**: `sc/instr/mode/level[/optdesc]/YYYY/MM/DD/`

where
* **sc**      = Spacecraft ID: {'mms1' | 'mms2' | 'mms3' | 'mms4'} 
* **instr**   = Instrument ID: many options
* **mode**    = Data rate mode: {'slow' | 'fast' | 'srvy' | 'brst'}
* **level**   = Data level: {'l1a', 'l1b', 'sitl', 'l2pre', 'l2', 'l3'}
* **optdesc** = Optional file name descriptor.
* **tstart**  = File start time (either `YYYYMMDD` or `YYYYMMDDhhmmss`)
* **X.Y.Z**   = Version number

The `start_date` and `end_date` parameters are half-inclusive. The API will find all files within the interval `[start_date, end_date)`. Note that `brst` files were not returned. This is because the time interval is such that only those files with start times of exactly `start_date` are returned. More often, `brst` files are scattered throughout the day.

From the SDC website, [How to Get Data](https://lasp.colorado.edu/mms/sdc/public/data/) provides a description of the services that the API provides. The [File Browser](https://lasp.colorado.edu/mms/sdc/public/data/) allows you to browse for datasets.The directory structure is similar to the file name format. [Datasets](https://lasp.colorado.edu/mms/sdc/public/datasets/) provides description of the instruments and data products.

### Download Data
We will set attributes to download level 2 vector magnetic field data from the FGM instrument. Level 2 data is the science quality data available to the public (no password required).

In [3]:
mysdc.sc = 'mms1'
mysdc.instr = 'fgm'
mysdc.mode = 'srvy'
mysdc.level = 'l2'
file = mysdc.Download()

62.4kk [03:09, 329k/s]                            


As you can see, the data was saved to the location specified by the `data_root` attribute using the SDC directory structure.

### Reading and Displaying Data
MMS data are saved into the Common Data Format [(CDF)](https://cdf.gsfc.nasa.gov/), created by NASA. SpacePy contains a module named [pycdf](https://pythonhosted.org/SpacePy/pycdf.html) that allows us to interface with the CDF files. As, the `pycdf` instructions state, we have to define the environment variable `CDF_LIB` so that it points to the newly installed CDF distribution.

In [4]:
# Import pycdf
import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf36_4-dist/lib/"
from spacepy import pycdf
import numpy as np
import matplotlib.dates
import matplotlib.pyplot as plt

# Open a file and list the variables
cdf = pycdf.CDF(file)
print(cdf)

# Read the magnetic field data in GSE coordinates
time = cdf['Epoch'][...]
b_gse = cdf['mms1_fgm_b_gse_srvy_l2'][...]
type(time[0])
np.shape(b_gse)

# Plot
plt.plot(time, b_gse)
plt.gcf().autofmt_xdate()
plt.gca().YLim([-100,100])
plt.show()

ModuleNotFoundError: No module named 'spacepy'